In [1]:
import os
import sys


In [2]:
%pwd


'/Users/yashdive/ML Repo/Portfolio Projects/poultry-disease-classifier/poultry-disease-classifier/research'

In [3]:
sys.path.append(os.path.abspath(os.path.join("..", "src")))


In [4]:
os.chdir("../")



In [5]:
%pwd

'/Users/yashdive/ML Repo/Portfolio Projects/poultry-disease-classifier/poultry-disease-classifier'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [23]:
from henalyze_CNN_classifier.constants import *
from henalyze_CNN_classifier.utils.common import read_yaml, create_directories


In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [26]:

import os
import urllib.request as request
import zipfile
from henalyze_CNN_classifier import logger
from henalyze_CNN_classifier.utils.common import get_size

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [40]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-07-15 18:26:16,887 - INFO - common - yaml file: config/config.yaml loaded successfully]
[2025-07-15 18:26:16,891 - INFO - common - yaml file: params.yaml loaded successfully]
[2025-07-15 18:26:16,893 - INFO - common - created directory at: artifacts]
[2025-07-15 18:26:16,894 - INFO - common - created directory at: artifacts/data_ingestion]
[2025-07-15 18:26:30,545 - INFO - 4037744573 - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B834:3BA0EA:DA756:1DA398:68764FF0
Accept-Ranges: bytes
Date: Tue, 15 Jul 2025 12:56:18 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd11000